In [37]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import pandas as pd
from pandas import ExcelWriter
from collections import Counter
import math

print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(level=2)
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(level=2)
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(level=2)
print("Initializing drivers ... done")
print()


Initializing drivers ... WS
Initializing drivers ... POS
Initializing drivers ... NER
Initializing drivers ... done



In [38]:
# Input text
fileName="富邦金（日盛）";
filename = "./_target/"+fileName+".xls"
result="./_target/"+fileName+"reslt.xls"
file = pd.read_excel(filename)
print(file.head(3))


      公開/公告號    公開/公告日          申請號       申請日  \
0  TWM614130  20210701  TW110203678  20210406   
1  TWM614128  20210701  TW110203633  20210401   
2  TWM614127  20210701  TW110203632  20210401   

                                                專利名稱  \
0  推薦名單產製系統SYSTEMS FOR GENERATING RECOMMENDATION ...   
1  適於支付保險賠款的電子裝置ELECTRONIC DEVICE SUITABLE FOR PA...   
2  適於分派理賠案件的電子裝置ELECTRONIC DEVICE SUITABLE FOR AS...   

                                                 申請人  \
0  富邦人壽保險股份有限公司; FUBON LIFE INSURANCE CO., LTD. (TW)   
1       富邦產物保險股份有限公司; FUBON INSURANCE CO., LTD. (TW)   
2       富邦產物保險股份有限公司; FUBON INSURANCE CO., LTD. (TW)   

                                                  摘要  \
0  ﻿一種推薦名單產製系統，系統包括一業務員資料庫、一登入模組、一名單模組、一第一客戶互動紀錄資...   
1  ﻿提供一種適於支付保險賠款的電子裝置。所述電子裝置包括儲存媒體、收發器以及處理器。儲存媒體儲...   
2  ﻿提供一種適於分派理賠案件的電子裝置。所述電子裝置包括收發器、儲存媒體以及處理器。儲存媒體儲...   

                                        IPC 引用專利  被參考次數      證書號  \
0  G06Q 30/02(2012.01); G06Q 40/00(2012.01)  NaN    NaN  M61

In [39]:
var=file["摘要"]
name=file["專利名稱"]
sort_key=[]
sort_words=[]
for i in range (len(file)) :
    text =[var[i]]
    sent_cnt=0
    
    self_list=[]## get combine word 
    common_list=[]## get the most common word 
    document=[]
    #Run pipeline
    #print("Running pipeline ... WS")
    ws = ws_driver(text)
    #print("Running pipeline ... POS")
    pos = pos_driver(ws)
    #print("Running pipeline ... NER")
    
    ner = ner_driver(text)
    
    print("Running pipeline ... done")
    
    print()
    def pack_ws_pos_sentece(sentence_ws, sentence_pos):
        assert len(sentence_ws) == len(sentence_pos)
        res = []
             
        ## 字詞的組合 根據屬性表將名詞-> 專有名詞 來增加字詞的特殊性
        pre_pos=''
        pre_ws=''
        for word_ws, word_pos in zip(sentence_ws, sentence_pos):
            ## 寫的好爛... 可以透過語言結構做優化
            res.append(f"{word_ws}({word_pos})")
            
            if(word_pos=="Na" or "V" in word_pos):
                common_list.append(word_ws)
                
            if((word_pos == "Na"or word_pos=="VC") and pre_pos!='' ):
            # use 
                Nb=f"{pre_ws}{word_ws}"
                #print(Nb)
                self_list.append(pre_ws+word_ws)
                #res.append(f"{Nb}(Nb)")
                pre_pos=""
                pre_ws=""
                    
            else :
                ## memorizing
                if(word_pos=="VC" or word_pos=="VD" or word_pos=="Na" or word_pos=="Nb"):
                    pre_pos=word_pos
                    pre_ws=word_ws
                    
                else :
                    pre_pos=''
                    pre_ws=''
                    #res.append(f"{word_ws}({word_pos})")
            if(word_pos=="PERIODCATEGORY" or word_pos=="SEMICOLONCATEGORY"):
                global sent_cnt,document
                sent_cnt+=1
                #print(word_ws+""+str(sent_cnt))
                
                document.append(res)
            
        return "\u3000".join(res)
    
    
    ## zip指標的標記
    for sentence, sentence_ws, sentence_pos, sentence_ner in zip(text, ws, pos, ner):
        #print(pos)
        pack_ws_pos_sentece(sentence_ws, sentence_pos)
    


    sort_key.append(Counter(self_list).most_common(10))

    sort_words.append(Counter(common_list).most_common(10))
    print(name[i])
    print("done")
    #print(sentence)
    print("------------")
#     print(sort_key[i])
#     print("------------")
#     print(sort_words[i])
#     for words in sort_key:
#         _TF=(words[1]/len(pos)) 
#         _IDF=math.log(words[1]/sent_cnt)
#         print(words[0])
#         print(_TF*_IDF)
#         print()
        #print(+str(TF_IDF))

    # for entity in sentence_ner:
    #     print(entity)
    print()
    del text
    del self_list,common_list,document 


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Running pipeline ... done

推薦名單產製系統SYSTEMS FOR GENERATING RECOMMENDATION LIST
done
------------



Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Running pipeline ... done

適於支付保險賠款的電子裝置ELECTRONIC DEVICE SUITABLE FOR PAYING INSURANCE INDEMNITIES
done
------------



Inference:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running pipeline ... done

線上機器人客服系統ONLINE ROBOT CUSTOMER SERVICE SYSTEM
done
------------



Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Running pipeline ... done

行動理財秘書系統Mobile financial management secretary system
done
------------



Inference: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]

Running pipeline ... done

雲端智能交易系統Cloud smart trading system
done
------------



In [40]:
## export back to the file 
bank ={"名稱":name,
        "KEY":sort_key,
        "WORD":sort_words
                }
print (pd.DataFrame(bank))

                                                    名稱  \
0    推薦名單產製系統SYSTEMS FOR GENERATING RECOMMENDATION ...   
1    適於支付保險賠款的電子裝置ELECTRONIC DEVICE SUITABLE FOR PA...   
2    適於分派理賠案件的電子裝置ELECTRONIC DEVICE SUITABLE FOR AS...   
3     職業代碼推薦系統NUMERICAL JOB CODE RECOMMENDATION SYSTEM   
4    基於區塊鏈之保險系統及其保險方法BLOCKCHAIN BASED INSURANCE SYS...   
..                                                 ...   
171                    股票健診系統Stock health check system   
172  智能推播服務系統與裝置SMART PUSHING SERVICE SYSTEM AND DE...   
173      線上機器人客服系統ONLINE ROBOT CUSTOMER SERVICE SYSTEM   
174  行動理財秘書系統Mobile financial management secretary ...   
175                 雲端智能交易系統Cloud smart trading system   

                                                   KEY  \
0    [(名單模組, 3), (推薦名單, 2), (客戶名單, 2), (歷史推薦, 2), (...   
1    [(儲存媒體, 3), (電子裝置, 2), (支付保險, 1), (處理器耦接, 1), ...   
2    [(理賠案件, 6), (儲存媒體, 3), (員工狀態檔, 3), (電子裝置, 2), ...   
3    [(職業代碼, 2), (資料輸出, 2), (推薦結果, 2), (推薦系統, 1), (...   
4    [(區塊鏈, 8

In [41]:
pd.DataFrame(bank).to_excel(result)

#pd.DataFrame(bank).to_excel(result)



<ipython-input-41-e3ba4ddf6bf6>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  pd.DataFrame(bank).to_excel(result)
